In [2]:
# Importing required Packages
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image 


In [12]:
# Read Loan Dataset
df = pd.read_csv("train.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Feature Engineering

### Create a new feature : Total_Income = Applicant Income + Coapplicant Income

In [46]:
df['Total_income'] = df['ApplicantIncome']+ df['CoapplicantIncome']

In [5]:
del df['ApplicantIncome']

In [6]:
del df['CoapplicantIncome']

In [15]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_income
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y,2900.0
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y,4106.0
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y,8312.0
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y,7583.0


### Create a new feature : Loan Amount to Total Income

In [ ]:
## Loan Amount to Total Income Ratio


In [16]:
df['LTI'] = (df['LoanAmount']/df['ApplicantIncome'])*100

In [21]:
df['LTI'].describe()

count    592.000000
mean       3.830616
std        5.582744
min        0.301614
25%        2.357604
50%        3.086484
75%        4.256322
max       90.000000
Name: LTI, dtype: float64

### Create a new feature : Income group by binning Total income <br>

**Condition :** <br>
<li>If total income is less than 5000 -------------------------> Low Income group <br>
<li>If total income is more than 5000 but less than 10000 -----> Medium Income group <br>
<li>If total income is more than 10000 ------------------------> High Income group

In [47]:
df['Total_income'].head()

0    5849.0
1    6091.0
2    3000.0
3    4941.0
4    6000.0
Name: Total_income, dtype: float64

In [60]:
def income_group(x):
    if x< 7000 : return " Low income_group"
    elif (x>7000 and x<10000) :return "meduim income group"
    else : return " High income group" 

In [62]:
df["Income_group"] = df["Total_income"].apply(income_group)
df["Income_group"]

0         Low income_group
1         Low income_group
2         Low income_group
3         Low income_group
4         Low income_group
              ...         
609       Low income_group
610       Low income_group
611    meduim income group
612    meduim income group
613       Low income_group
Name: Income_group, Length: 614, dtype: object

In [32]:
df["Total_income"].dtype

dtype('float64')

### Create a new feature : Stable group by using Multiple columns <br>

**Condition :** <br>
<li>If a applicant is married,graduate and not self employed ----> Stable <br>
<li> Else  ---------------------------------------------------------------> Unstable 

In [71]:
def stable_group(data):
    
    if(data["Married"] == "Yes") and (data['Education']=='Graduation') and (data["Self_Employed"]== "NO"):
        
        return "Stable_Group"
    
    else : return " Unstable Group"
    

    

In [72]:
df['Stable_group'] = df.apply(stable_group, axis=1)

In [79]:
df[['Married',"Education",'Self_Employed']]

,Married,Education,Self_Employed
0,No,Graduate,No
1,Yes,Graduate,No
2,Yes,Graduate,Yes
3,Yes,Not Graduate,No
4,No,Graduate,No
...,...,...,...
609,No,Graduate,No
610,Yes,Graduate,No
611,Yes,Graduate,No
612,Yes,Graduate,No


## Group By

A groupby operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.

### Load data and assign it to a variable called drinks.


In [80]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv'

In [81]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

drinks = pd.read_csv(url)

In [83]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


In [84]:
drinks['continent'].unique()

array(['AS', 'EU', 'AF', nan, 'SA', 'OC'], dtype=object)

In [ ]:
#Frequency of Observation in each continent


In [85]:
drinks['continent'].value_counts()

AF    53
EU    45
AS    44
OC    16
SA    12
Name: continent, dtype: int64

In [90]:
drinks.groupby['continent'].wine_servings.min()

TypeError: 'method' object is not subscriptable

### Inputs Parameters<br><br>

<li><b>Primary key column name :</b><br>
<li><b>Target column :</b> <br>
<li><b>agg func :</b> min ; max ; mean ; median etc<br>

### Which continent drinks more beer on average?

In [102]:
drinks.groupby('continent').wine_servings.mean()

continent
AF     16.264151
AS      9.068182
EU    142.222222
OC     35.625000
SA     62.416667
Name: wine_servings, dtype: float64

### Which continent drinks minimum wine ?

In [93]:
drinks.groupby('continent').wine_servings.min()

continent
AF    0
AS    0
EU    0
OC    0
SA    1
Name: wine_servings, dtype: int64

### Which continent drinks max alcohol ?

In [99]:
drinks.groupby('continent').wine_servings.max()

continent
AF    233
AS    123
EU    370
OC    212
SA    221
Name: wine_servings, dtype: int64

## Join two dataframes 

### Using Concatenate function

In [103]:
Div_1 = pd.DataFrame({
        'roll_no': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']})


Div_2 = pd.DataFrame({
        'roll_no': ['6', '7', '8', '9', '10'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']})

In [104]:
Div_1

,roll_no,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [ ]:
div

In [105]:
Div_2

,roll_no,first_name,last_name
0,6,Billy,Bonder
1,7,Brian,Black
2,8,Bran,Balwner
3,9,Bryce,Brice
4,10,Betty,Btisan


In [108]:
pd.concat([Div_1,Div_1],axis =0)

,roll_no,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


#### Join both the tables on top of each other

### New Data

#### Join both the subjects marks

In [109]:
English = pd.DataFrame({
        'roll_no': ['2', '1', '3', '4', '5','6'],
#         'name': ['Amy', 'Alex', 'Allen', 'Alice', 'Ayoung','Rahul'], 
        'english': [56,78,65,89,94,88]})


Maths = pd.DataFrame({
        'roll_no': ['1', '2', '3', '4', '5','7'],
#         'name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung','Ganesh'], 
        'maths': [65,26,78,59,75,87]})

In [110]:
English

,roll_no,english
0,2,56
1,1,78
2,3,65
3,4,89
4,5,94
5,6,88


In [111]:
Maths

,roll_no,maths
0,1,65
1,2,26
2,3,78
3,4,59
4,5,75
5,7,87


In [114]:
pd.concat([English, Maths['maths']], axis=1)

,roll_no,english,maths
0,2,56,65
1,1,78,26
2,3,65,78
3,4,89,59
4,5,94,75
5,6,88,87


## Merge

### Inputs Parameters<br><br>

<li><b>Left dataframe :</b> 1st Dataframe <br>
<li><b>Right dataframe :</b> 2nd Dataframe <br>
<li><b>on :</b> Primary key<br>
<li><b>how :</b> inner/outer/left/right<br>


In [113]:
Image('joins.png')

FileNotFoundError: No such file or directory: 'joins.png'

FileNotFoundError: No such file or directory: 'joins.png'

<IPython.core.display.Image object>